In [1]:
#Verify necessary packages installed
# !pip install transformers evaluate datasets requests pandas xlrd scikit-learn torch tensorflow openpyxl

import pandas as pd, transformers, openpyxl, random
from tqdm import tqdm

In [2]:
#Select a model checkpoint. For my purposes, training on 3080 Ti at home with 12GB RAM
#Will select smaller model because I don't want to overburden my GPU =)
#Models
#Checkpoint name	Num layers	Num parameters
#esm2_t48_15B_UR50D	48	15B
#esm2_t36_3B_UR50D	36	3B
#esm2_t33_650M_UR50D	33	650M
#esm2_t30_150M_UR50D	30	150M
#esm2_t12_35M_UR50D	12	35M
#esm2_t6_8M_UR50D	6	8M

model_checkpoint = "facebook/esm2_t12_35M_UR50D"

mass_spec_data = "../data/NIHMS1541512-supplement-Sup_Data1.xlsx"

filter_cell_line_peptides = "../data/NIHMS1541512-supplement-Sup_Tab1.xlsx" #table e
filter_patient_peptides = "../data/NIHMS1541512-supplement-Sup_Tab6.xlsx" #table a


In [3]:
#Create peptide filtration dict
#Actually, these look like filtered-in data points, based on Suppl. 1d metrics
filtration_dict = {}

filter_in_df = pd.read_excel(filter_cell_line_peptides, engine="openpyxl", sheet_name="e. Filtered peptide list")
alleles = list(filter_in_df['Allele'].drop_duplicates())

#populate sets
for allele in alleles:
    filtration_dict[allele] = set(filter_in_df.query("Allele == @allele")['Peptide'])


df2 = pd.read_excel(filter_patient_peptides, engine="openpyxl", sheet_name="a. Filtered peptide list") 

#filter patient peptides ubiquitously 
filtration_dict["patient_data"] = set(df2['Peptide'])

In [4]:
# # This was operating under the assumption that the filtration list was filter-out,
# # when it is in fact filter-in
# retained_neoepitopes={}
# filtered_counts={} 
# for allele in alleles:
#     allele_df = pd.read_excel(mass_spec_data, engine="openpyxl", sheet_name=allele)
#     print(len(allele_df))
#     filtered_allele_df = allele_df.query(
#         "sequence not in @filtration_dict[@allele]")# and sequence not in @filtration_dict['patient_data']") #This is just monoallelic data
#     filtered_counts[allele] = len(allele_df) - len(filtered_allele_df)
#     print(f"{str(filtered_counts[allele])} neoepitopes filtered from {allele}; {len(filtered_allele_df)} retained.")
#     retained_neoepitopes[allele] = set(filtered_allele_df['sequence'])


In [5]:
#This is already in the correct format. It's always fun to waste effort!
#For simplicity, will exclude HLA-G
filter_in_df = filter_in_df.loc[~filter_in_df.Allele.apply(lambda x: x.startswith('G'))].copy()
filter_in_df.loc[filter_in_df.Length > 11].groupby('Length').count()

,Allele,Peptide
Length,,
12,5177,5177
13,2541,2541
14,1027,1027
15,590,590
16,433,433
17,377,377
18,294,294
19,241,241
20,173,173


In [6]:
#Get size of negatives we need to sample
sample_sizes = (filter_in_df.groupby("Length").count()['Allele']*1000).to_dict()

In [7]:
import urllib.request
from Bio import SeqIO

#Get fasta files, parse for alleles of interest
#For simplicity of demonstration, will not include HLA-G
hla_a_file="https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/fasta/A_prot.fasta"
hla_b_file="https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/fasta/B_prot.fasta"
hla_c_file="https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/fasta/C_prot.fasta"
#hla_g_file="https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/fasta/G_prot.fasta"

allele_mapper = {}

hla_files = [hla_a_file, hla_b_file, hla_c_file]#, hla_g_file]
#Loop through fasta files, get record of each HLA and add it to mapper if not present
for hla_file in hla_files:
    output_file = hla_file.split('/')[-1]
    write_dir = f"../data/{output_file}"
    urllib.request.urlretrieve(hla_file, write_dir)
    fasta_sequences = SeqIO.parse(open(write_dir),'fasta')
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        description = fasta.description
        allele = description.split(" ")[1]

        allele_name = ''.join(allele.split(':')[:2]).replace('*', '')
        allele_suffix = ''.join(allele.split(':')[2:])

#        if ((allele_suffix in ['0101', '01', '']) and (allele_name not in allele_mapper)):
        #Naively assume that first entry is best entry. I'm sure this'll come back to bite me
        if ((allele_name in list(filter_in_df['Allele'].drop_duplicates())) and
            (allele_name not in allele_mapper)): 
            allele_mapper[allele_name] = sequence
        else:
            continue

assert(len(allele_mapper) == len(filter_in_df.Allele.drop_duplicates())) 

In [34]:
#Get human proteome for negative sampling
import gzip 
import random
from tqdm import tqdm

#TODO: REDO THIS CELL, TAKES TOO LONG TO RUN

def get_random_str(main_str, substr_len):
    #https://stackoverflow.com/questions/58334846/randomly-extracting-substrings-of-equal-length-from-the-main-string
    idx = random.randrange(0, len(main_str) - substr_len + 1)    # Randomly select an "idx" such that "idx + substr_len <= len(main_str)".
    return main_str[idx : (idx+substr_len)]

proteome_link = "https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz"
urllib.request.urlretrieve(proteome_link,
                           '../data/UP000005640_9606.fasta.gz') 

#Determine length of proteome
proteome_length = 0
with gzip.open('../data/UP000005640_9606.fasta.gz', 'rt') as handle:
    proteine_sequences = SeqIO.parse(handle, 'fasta')
    for fasta in proteine_sequences:
        proteome_length +=1

#Create 1000 negatives for every positive
#This is very loopy and is expected to take a while...
proteome_dict = {}
with gzip.open('../data/UP000005640_9606.fasta.gz', 'rt') as handle:
    protein_sequences = SeqIO.parse(handle, 'fasta')
    for i, fasta in enumerate(tqdm(protein_sequences)):
        for sample_length, sample_size in sample_sizes.items(): #Determine how many samples needed from each protein
            if sample_length not in proteome_dict:
                proteome_dict[sample_length] = set()
            name, sequence = fasta.id, str(fasta.seq)
            if len(proteome_dict[sample_length]) >= sample_size:
                continue #no samples needed here
            #If < 1 needed from each, just sample 3 until we have enough
            samples_needed_from_sequence = int(max((sample_size/proteome_length), 3))
            #Verify no overlap with binding alleles, conservatively
            new_protein_set = set()
            for sample in range(0, samples_needed_from_sequence):
                try:
                    new_protein_set.add(get_random_str(sequence, sample_length))
                except:
                    break #Just move on, not worth resolving for a negative sampler

            new_protein_set = new_protein_set - set(filter_in_df['Peptide'])            
            proteome_dict[sample_length] = new_protein_set.union(proteome_dict[sample_length])
            
            

20654it [10:41:22,  1.86s/it]


In [38]:
# import pickle

# #dump pickle so I don't have to do previous step again (for now)
# with open("../data/negative_samples.pkl", 'wb') as f:
#     pickle.dump(proteome_dict, f)

# backup_df = filter_in_df.copy()

In [9]:
import pickle

with open ("../data/negative_samples.pkl", "rb") as f:
    proteome_dict = pickle.load(f)

backup_df = filter_in_df.copy()

In [15]:
filter_in_df = backup_df.copy()


negs = 5
filter_in_df['label'] = 1

#Create `negs` negatives for every positive
for i, allele in enumerate(tqdm(list(filter_in_df.Allele.drop_duplicates()))):
    sub_df = filter_in_df.query('Allele == @allele').copy()
    for length in list(sub_df['Length'].drop_duplicates()):
        sub_df_by_len = sub_df.query('Length == @length')
        use_int = negs * len(sub_df_by_len)
        add_these = pd.DataFrame({'Allele': [allele]*use_int,
                     'Length': [length]*use_int, 
                     'Peptide': random.sample(proteome_dict[length], use_int),
                     'label': [0]*use_int})
        filter_in_df = filter_in_df.append(add_these, ignore_index=True)

filter_in_df.reset_index(drop=True, inplace=True)

filter_in_df['allele_sequence'] = filter_in_df['Allele'].map(allele_mapper)
filter_in_df['predict_on'] = filter_in_df['allele_sequence'] + filter_in_df['Peptide'] 
filter_in_df['predict_on_len'] = filter_in_df['predict_on'].apply(lambda x: len(x))

                     

100%|███████████████████████████████████████████████████████████████████████████████████| 92/92 [03:15<00:00,  2.13s/it]


In [16]:
# #dump pickle so I don't have to do previous step again (for now)
# with open("../data/training_samples_5prev.pkl", 'wb') as f:
#     pickle.dump(filter_in_df, f)

# filter_in_df

,Allele,Length,Peptide,label,allele_sequence,predict_on,predict_on_len
0,A0101,8,ADMGHLKY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373
1,A0101,8,ELDDTLKY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373
2,A0101,8,FSDNIEFY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373
3,A0101,8,FTELAILY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373
4,A0101,8,GLDEPLLK,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373
...,...,...,...,...,...,...,...
1105093,C1701,39,ALELVTEKGHTFAEELQKIQCTLQDVGSALATPCSSARE,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,411
1105094,C1701,39,LKDLEKWQNNLLPSRQFGFIVLTTSAGIMDHEEARRKHT,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,411
1105095,C1701,39,QKNFLSYDCGSDKVLSMGHLEEQLYATDAWGKQLEMLRE,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,411
1105096,C1701,39,PEEKGNIVEEQPQKNTTEKLGVSAPTIPVRRRLAWDTEN,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,411


In [8]:
import pickle

with open ("../data/training_samples_5prev.pkl", "rb") as f:
    filter_in_df = pickle.load(f)

backup_df = filter_in_df.copy()

In [26]:
#Need training data to have data points from all alleles!
filter_in_df.sort_values('Allele', inplace=True)
filter_in_df.reset_index(drop=True,inplace=True)
filter_in_df['training']=1 #training == 1, test == 2
for i, allele in enumerate(tqdm(list(filter_in_df.Allele.drop_duplicates()))):
    filter_in_df.loc[filter_in_df.Allele == allele,'training'] = filter_in_df.loc[
    filter_in_df.Allele == allele,'training'].apply(lambda x: x + random.choices([0, 1], weights=(75, 25))[0])

100%|███████████████████████████████████████████████████████████████████████████████████| 92/92 [00:08<00:00, 11.37it/s]


In [30]:
print(len(filter_in_df.loc[filter_in_df.training==1]))
print(len(filter_in_df.loc[filter_in_df.training==2]))
filter_in_df

829159
275939


,Allele,Length,Peptide,label,allele_sequence,predict_on,predict_on_len,training
0,A0101,8,ADMGHLKY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373,1
1,A0101,11,PEKIAVEIPKR,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,2
2,A0101,9,GHQVALSSI,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,374,1
3,A0101,11,KQKEVFLPSTP,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1
4,A0101,11,LLSNSSSLWRS,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1
...,...,...,...,...,...,...,...,...
1105093,C1701,9,HSVPPVSRK,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,2
1105094,C1701,9,HPTDPTVLI,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1
1105095,C1701,9,KKKVVFCPV,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1
1105096,C1701,9,SEADPQALL,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1


In [31]:
#dump pickle so I don't have to do previous step again (for now)
with open("../data/training_samples_5prev.pkl", 'wb') as f:
    pickle.dump(filter_in_df, f)

filter_in_df

,Allele,Length,Peptide,label,allele_sequence,predict_on,predict_on_len,training
0,A0101,8,ADMGHLKY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373,1
1,A0101,11,PEKIAVEIPKR,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,2
2,A0101,9,GHQVALSSI,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,374,1
3,A0101,11,KQKEVFLPSTP,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1
4,A0101,11,LLSNSSSLWRS,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1
...,...,...,...,...,...,...,...,...
1105093,C1701,9,HSVPPVSRK,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,2
1105094,C1701,9,HPTDPTVLI,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1
1105095,C1701,9,KKKVVFCPV,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1
1105096,C1701,9,SEADPQALL,0,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,381,1


In [32]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

training_df = filter_in_df.loc[filter_in_df.training == 1].copy()
testing_df = filter_in_df.loc[filter_in_df.training == 2].copy()

#Generate data
train_sequences = training_df['predict_on'].tolist()
train_labels = training_df['label'].tolist()

test_sequences = testing_df['predict_on'].tolist()
test_labels = testing_df['label'].tolist()

#Split data
#print("Splitting data...")
#train_sequences, test_sequences, train_labels, test_labels = train_test_split(training_sequences, training_labels, test_size=0.25, shuffle=True)

del filter_in_df #Hopefully free some memory

#Tokenize data
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print("Tokenizing testing data...")
test_tokenized = tokenizer(test_sequences)

print("Tokenizing training data...")
train_tokenized = tokenizer(train_sequences)



Tokenizing testing data...
Tokenizing training data...


In [33]:
with open("../data/training_samples_5prev_traintest_split.pkl", 'wb') as f:
    pickle.dump([train_tokenized, test_tokenized, train_labels, test_labels], f)

 

In [1]:
import pickle

with open("../data/training_samples_5prev_traintest_split.pkl", 'rb') as f:
    train_tokenized, test_tokenized, train_labels, test_labels = pickle.load(f)



In [2]:
#Setup for PyTorch
from datasets import Dataset

print("Creating test dset...")

#test_dset = Dataset.from_dict(test_tokenized)
#test_dset = test_dset.add_column("labels", test_labels)

print("Dumping test dset...")

#with open("../data/test_dset_prev5.pkl", 'wb') as f:
#    pickle.dump(test_dset, f)

del(test_tokenized)
del(test_labels)
#del(test_dset)

print("Creating train dset...")

train_dset = Dataset.from_dict(train_tokenized)
train_dset = train_dset.add_column("labels", train_labels)

print("Dumping train dset...")

with open("../data/train_dset_prev5.pkl", 'wb') as f:
    pickle.dump(train_dset, f)

train_dset

Creating test dset...
Dumping test dset...
Creating train dset...
Dumping train dset...


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 829159
})

In [4]:
#Let's try training something now... Oh boy



Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 828823
})

In [ ]:
import numpy as np

"""
Create two matrices matrix. For HLA-A, matrix will be 365x30 
(length of HLA-AxNumber of HLA-A serotypes -1). We need to 
identify 30-50 amino acids that exhibit the greatest 
variety. In this scenario, pairwise comparisons between each
2 HLAs, write down proportion of overlap between two

While doing the above, create pair-wise vectors, 
365 long. 0-1 whether identical or not. At the end,
sum all 30 vectors to determine sites of interest
"""

iterate_alleles = list(allele_mapper.keys())
pairwise_vectors = {}

for i, allele in enumerate(iterate_alleles):
    key_a = allele
    for key_b in iterate_alleles:
        if key_a[0] != key_b[0]: #Only compare HLA-As to HLA-As, etc
            continue
        allele_key = "|".join(sorted([key_a, key_b]))
        if allele_key in pairwise_vectors:
            continue
    
        a1 = allele_mapper[key_a]
        a2 = allele_mapper[key_b]

        pairwise_vectors[allele_key] = []

        len_a1 = len(a1)
        len_a2 = len(a2)
        use_len = min(len_a1, len_a2)

        for idx, aa in enumerate(a1):
            if idx < use_len:
                if aa == a2[idx]:
                    pairwise_vectors[allele_key].append(0)
                else:
                    pairwise_vectors[allele_key].append(1)

        pairwise_vectors[allele_key] = np.array(pairwise_vectors[allele_key])

#Get 50-mer index for greatest number of alleles that differ pair-wise
A_alleles = []
B_alleles = []
C_alleles = []
G_alleles = []
for key, value in pairwise_vectors.items():
    if key.startswith('A'):
        A_alleles.append(value)
    elif key.startswith('B'):
        B_alleles.append(value)
    elif key.startswith('C'): 
        C_alleles.append(value)
    elif key.startswith('G'):
        G_alleles.append(value)

A_alleles = np.array(A_alleles)
B_alleles = np.array(B_alleles)
C_alleles = np.array(C_alleles)
G_alleles = np.array(G_alleles) 

In [ ]:
A_alleles = []
B_alleles = []
C_alleles = []
G_alleles = []
for key, value in pairwise_vectors.items():
    if key.startswith('A'):
        A_alleles.append(value)
    elif key.startswith('B'):
        B_alleles.append(value)
    elif key.startswith('C'): 
        C_alleles.append(value)
    elif key.startswith('G'):
        G_alleles.append(value)

A_alleles = np.array(A_alleles)
B_alleles = np.array(B_alleles)
C_alleles = np.array(C_alleles[:-1])
G_alleles = np.array(G_alleles)

def get_indices_of_highest_values(input_array, return_n_indices):
    return(sorted(range(len(input_array)), key = lambda sub: input_array[sub])[-return_n_indices:])

A_indices = sorted(get_indices_of_highest_values(sum(A_alleles), 50))
B_indices = sorted(get_indices_of_highest_values(sum(B_alleles), 50))
C_indices = sorted(get_indices_of_highest_values(sum(C_alleles), 50))

#get_indices_of_highest_values(A_alleles, 50)

In [ ]:
len([x for x in sum(A_alleles) if x != 0])

In [ ]:
len([x for x in sum(B_alleles) if x != 0])

In [ ]:
len([x for x in sum(C_alleles[:-1]) if x != 0])

In [ ]:
A_indices = sorted(get_indices_of_highest_values(sum(A_alleles), 50))
B_indices = sorted(get_indices_of_highest_values(sum(B_alleles), 50))
C_indices = sorted(get_indices_of_highest_values(sum(C_alleles), 50))